In [11]:
import smtplib
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import yagmail
import schedule
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import http.client
import json

In [12]:
def fetch_jooble_jobs1(api_key, job_titles2=["Graphic Designer", "Digital Marketing Specialist", "Public Relations Specialist"],limit = 20, location="New York"):
    # Jooble API host
    host = 'jooble.org'

    # List to store all job results
    all_jobs = []

    # Loop through each job title
    for job_title in job_titles2:
        # Set up the body of the request (JSON query)
        body = json.dumps({
            "keywords": job_title,
            "location": location
        })

        # Create an HTTP connection
        connection = http.client.HTTPConnection(host)

        # Set up headers
        headers = {
            "Content-type": "application/json",
            "Authorization": f"Bearer {api_key}"  # Use your API key here
        }

        try:
            # Send the request
            connection.request('POST', f'/api/{api_key}', body, headers)
            response = connection.getresponse()


            # Check if the request was successful
            if response.status == 200:
                print("API Response Status:", response.status)
                response_data = response.read().decode('utf-8')  # Decode the response bytes to string

                # Try to parse the response JSON
                data = json.loads(response_data)
                jobs = data.get("jobs", [])

                # Debugging: Print the entire response to check the structure
                print("Full Response Data:", json.dumps(data, indent=4))

                # Parse and store the job results
                for job in jobs:
                    # Fetch the job title, description, company, location, and link
                    title = job.get("title", "").strip()  # Job title field updated
                    if not title:  # If title is empty, skip this job
                        title = "No Title"  # Replace empty title with a default text
                    company = job.get("company", "No Company")
                    location = job.get("location", "No Location")
                    link = job.get("link", "No Link")
                    snippet = job.get("snippet", "No Description")  # Description field

                    # Clean the description by removing HTML tags using BeautifulSoup
                    cleaned_snippet = BeautifulSoup(snippet, "html.parser").get_text()

                    # Display the job info (including description)
                    all_jobs.append(f"{title} at {company} - {location}: {link}\nDescription: {cleaned_snippet}\n")

            else:
                print(f"Error fetching jobs for {job_title}: {response.status} - {response.reason}")
                print("Full Response:", response.read().decode('utf-8'))  # Print the full response for more insights

        except Exception as e:
            print(f"Error during API request for {job_title}: {e}")
        if len(all_jobs) >= limit:
            break

    return all_jobs

# Use the function to fetch jobs
api_key = "288a2c27-66a8-443a-9028-722bab20f3aa"  # Your API key
job_titles = ["Graphic Designer", "Digital Marketing Specialist", "Public Relations Specialist"]  # List of job titles to search for
jobs = fetch_jooble_jobs1(api_key, job_titles)

# Print the fetched jobs
if jobs:
    for job in jobs:
        print(job)
else:
    print("No jobs found.")






API Response Status: 200
Full Response Data: {
    "totalCount": 9540,
    "jobs": [
        {
            "title": "Graphic Designer",
            "location": "New York, NY",
            "snippet": "<b>Graphic Designer </b>\r\n JobDiva has an excellent opportunity for an enthusiastic Graphic Designer with a high design esthetic and original vision to collaborate with copywriters, video, and development teams. This position reports directly into CMO. \r\n \r\n \r\n bout the company...&nbsp;",
            "salary": "",
            "source": "appcast.io",
            "type": "",
            "link": "https://jooble.org/jdp/2736575105885760048",
            "company": "JobDiva",
            "updated": "2024-12-17T11:03:00.4146804+00:00",
            "id": 2736575105885760048
        },
        {
            "title": "Senior Graphic Designer & Illustrator",
            "location": "Brooklyn, NY",
            "snippet": "&nbsp;...Graza is looking for a talented <b>graphic designer </b>and il

In [13]:
def send_email(job_links):
    sender_email = "angelpeters15@gmail.com"
    app_password = "xgaa umyz mxzx wowg"  # Use an App Password
    receiver_email = "liza.peters1@yahoo.com"
    receiver_email2 = "misselectro101@gmail.com"

    
    subject = "Morning Job Listings"
    body = "Good Morning Liza, Here are the latest job openings for Graphic Design:\n\n" + "\n".join(job_links)
    
    yag = yagmail.SMTP(sender_email, app_password)
    yag.send(to=receiver_email, subject=subject, contents=body)
    print("Email sent successfully!")

In [14]:
def job_search():
    # Make sure to pass the API key when calling fetch_jooble_jobs()
    api_key = "288a2c27-66a8-443a-9028-722bab20f3aa"  # Your API key
    jobs = fetch_jooble_jobs1(api_key)  # Pass the api_key to the function
    send_email(jobs)  # Send the fetched jobs via email


In [15]:
job_search()

API Response Status: 200
Full Response Data: {
    "totalCount": 9540,
    "jobs": [
        {
            "title": "Graphic Designer",
            "location": "New York, NY",
            "snippet": "<b>Graphic Designer </b>\r\n JobDiva has an excellent opportunity for an enthusiastic Graphic Designer with a high design esthetic and original vision to collaborate with copywriters, video, and development teams. This position reports directly into CMO. \r\n \r\n \r\n bout the company...&nbsp;",
            "salary": "",
            "source": "appcast.io",
            "type": "",
            "link": "https://jooble.org/jdp/2736575105885760048",
            "company": "JobDiva",
            "updated": "2024-12-17T11:03:00.4146804+00:00",
            "id": 2736575105885760048
        },
        {
            "title": "Senior Graphic Designer & Illustrator",
            "location": "Brooklyn, NY",
            "snippet": "&nbsp;...Graza is looking for a talented <b>graphic designer </b>and il